In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plp
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import ttest_rel, ttest_ind
from scipy.stats.contingency import association
from scipy.stats import spearmanr, pearsonr
pd.set_option('display.max_columns', None)
from scipy.stats import f_oneway, norm, f, chi2, chi2_contingency
import statsmodels.api as sm
from statsmodels.formula.api import ols
scaler = MinMaxScaler()

In [3]:
df_cars=pd.read_csv('C:/Users/Usuario/Documents/Inhale-the-present-exhale-the-past.-/Data/Clean_data/Matriculaciones.csv', encoding= 'utf-8',sep=';')
df_budget_mad=pd.read_csv('C:/Users/Usuario/Documents/Inhale-the-present-exhale-the-past.-/Data\Clean_data/Presupuestos_madrid.csv', encoding='utf-8')
df_budget_sp=pd.read_csv('C:/Users/Usuario/Documents/Inhale-the-present-exhale-the-past.-/Data/Clean_data/Presupuestos.csv', encoding='latin1',sep=';')


In [9]:
df_budget_mad.head()
df_budget_mad.rename(columns={'Año':'year','Id Política':'policy','Nombre Política':'policy_name','Id Programa':'program_id',
                          'Nombre Programa':'program_name','Presupuesto Gastos':'spending_budget','Gastos Reales':'total_spent'},
                            inplace=True)

In [16]:
df_budget_sp.dropna(inplace=True)

In [22]:
df_budget_sp = df_budget_sp.applymap(lambda x: x.lower() if type(x) == str else x)

In [23]:
df_budget_sp.head()

,ESTADO DE EJECUCIÓN DEL PRESUPUESTO DE GASTOS,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
3,centro presupuestario,descripción centro presupuestario,sección,descripción sección,centro gestor,descripción centro gestor,programa,descripción programa,capítulo,descripción capítulo,crédito inicial,crédito actual,mod. crédito,autorizado,saldo cdto. pto.,dispuesto,saldo autoriz.,obligaciones,saldo disposic.
4,1001,admón.comunidad de madrid,03,presidencia de la comunidad de madrid,030010000,presidencia de la comunidad de madrid,912m,asesoramiento a la presidencia,1,gastos de personal,"1.557.887,00","1.532.495,25","-25.391,75","1.137.782,85","394.712,40","1.137.782,85",0,"1.137.782,85",0
5,1001,admón.comunidad de madrid,03,presidencia de la comunidad de madrid,030010000,presidencia de la comunidad de madrid,912m,asesoramiento a la presidencia,2,gastos corrientes en bienes y servicios,"33.295,00","63.295,00","30.000,00","52.417,71","10.877,29","52.417,71",0,"52.417,71",0
6,1001,admón.comunidad de madrid,03,presidencia de la comunidad de madrid,030010000,presidencia de la comunidad de madrid,912m,asesoramiento a la presidencia,8,activos financieros,"1.366,00","1.366,00",0,0,"1.366,00",0,0,0,0
7,1001,admón.comunidad de madrid,04,"cultura,turismo y deporte",040010000,"s.g.t.cultura,turismo y deporte",331m,"dirección y gest.admin.cultura,turismo y deporte",1,gastos de personal,"13.213.315,00","14.020.962,81","807.647,81","10.513.178,09","3.507.784,72","10.513.178,09",0,"10.377.297,24","135.880,85"


In [25]:
df_budget_sp.rename(columns={'ESTADO DE EJECUCIÓN DEL PRESUPUESTO DE GASTOS':'budget_center',
                             'Unnamed: 1':'center_desc','Unnamed: 2':'sector','Unnamed: 3':'sector_desc',
                             'Unnamed: 4':'managment_center','Unnamed: 5':'managment_center_desc','Unnamed: 6':'program',
                             'Unnamed: 7':'program_desc','Unnamed: 8':'chapter','Unnamed: 9 ':'chapter_desc',
                             'Unnamed: 10':'initial_credit','Unnamed: 11':'actual_credit','Unnamed: 12':'credit_mod',
                             'Unnamed: 13':'approved_budget','Unnamed: 14':'balance','Unnamed: 15':'allocated_budget',
                             'Unnamed: 16':'authorized_budget','Unnamed: 17':'obligations','Unnamed: 18':'operable_budget'},
                             inplace=True)

In [27]:
df_budget_sp.head()

,budget_center,center_desc,sector,sector_desc,managment_center,managment_center_desc,program,program_desc,chapter,Unnamed: 9,initial_credit,actual_credit,credit_mod,approved_budget,balance,allocated_budget,authorized_budget,obligations,operable_budget
3,centro presupuestario,descripción centro presupuestario,sección,descripción sección,centro gestor,descripción centro gestor,programa,descripción programa,capítulo,descripción capítulo,crédito inicial,crédito actual,mod. crédito,autorizado,saldo cdto. pto.,dispuesto,saldo autoriz.,obligaciones,saldo disposic.
4,1001,admón.comunidad de madrid,03,presidencia de la comunidad de madrid,030010000,presidencia de la comunidad de madrid,912m,asesoramiento a la presidencia,1,gastos de personal,"1.557.887,00","1.532.495,25","-25.391,75","1.137.782,85","394.712,40","1.137.782,85",0,"1.137.782,85",0
5,1001,admón.comunidad de madrid,03,presidencia de la comunidad de madrid,030010000,presidencia de la comunidad de madrid,912m,asesoramiento a la presidencia,2,gastos corrientes en bienes y servicios,"33.295,00","63.295,00","30.000,00","52.417,71","10.877,29","52.417,71",0,"52.417,71",0
6,1001,admón.comunidad de madrid,03,presidencia de la comunidad de madrid,030010000,presidencia de la comunidad de madrid,912m,asesoramiento a la presidencia,8,activos financieros,"1.366,00","1.366,00",0,0,"1.366,00",0,0,0,0
7,1001,admón.comunidad de madrid,04,"cultura,turismo y deporte",040010000,"s.g.t.cultura,turismo y deporte",331m,"dirección y gest.admin.cultura,turismo y deporte",1,gastos de personal,"13.213.315,00","14.020.962,81","807.647,81","10.513.178,09","3.507.784,72","10.513.178,09",0,"10.377.297,24","135.880,85"


In [40]:
df_budget_sp= df_budget_sp.drop(df_budget_sp.index[0])
df_budget_sp.head()

,budget_center,center_desc,sector,sector_desc,managment_center,managment_center_desc,program,program_desc,chapter,Unnamed: 9,initial_credit,actual_credit,credit_mod,approved_budget,balance,allocated_budget,authorized_budget,obligations,operable_budget
1,1001,admón.comunidad de madrid,04,"cultura,turismo y deporte",040100000,d.g.promoción cultural,333a,museos y exposiciones,2,gastos corrientes en bienes y servicios,"7.025.720,00","7.275.720,00","250.000,00","6.010.660,29","1.265.059,71","6.008.394,02","2.266,27","4.623.230,02","1.385.164,00"
2,1001,admón.comunidad de madrid,04,"cultura,turismo y deporte",040100000,d.g.promoción cultural,333a,museos y exposiciones,4,transferencias corrientes,"1.399.229,00","1.394.229,00","-5.000,00","435.000,00","959.229,00","134.500,00","300.500,00","5.000,00","129.500,00"
3,1001,admón.comunidad de madrid,04,"cultura,turismo y deporte",040100000,d.g.promoción cultural,333a,museos y exposiciones,6,inversiones reales,"986.080,00","992.231,64","6.151,64","797.288,10","194.943,54","797.288,10",0,"446.436,29","350.851,81"
4,1001,admón.comunidad de madrid,04,"cultura,turismo y deporte",040100000,d.g.promoción cultural,334a,promoción y difusión cultural,1,gastos de personal,"3.250.954,00","3.283.793,54","32.839,54","2.308.044,32","975.749,22","2.308.044,32",0,"2.256.693,61","51.350,71"
5,1001,admón.comunidad de madrid,04,"cultura,turismo y deporte",040100000,d.g.promoción cultural,334a,promoción y difusión cultural,2,gastos corrientes en bienes y servicios,"22.595.922,00","22.922.367,64","326.445,64","17.952.455,47","4.969.912,17","17.952.455,46","0,01","13.480.419,61","4.472.035,85"


In [39]:

df_budget_sp.reset_index(drop=True, inplace=True)




In [42]:
df_budget_sp.to_csv('df_budget_sp.csv',index=False)
df_budget_mad.to_csv('df_budget_mad.csv',index=False)